# Enrichment of regulons from Regulons_by_strain

In [1]:
import os
import sys
from importlib import reload

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)

import pandas as pd
from IPython.display import display

from orangecontrib.bioinformatics.geneset.__init__ import list_all

import jupyter_functions as jf
reload(jf)
import jupyter_functions as jf
from functionsDENet import loadPickle, savePickle
import stages_DE.enrichment_library as enr
reload(enr)
import stages_DE.enrichment_library as enr

In [2]:
#Path for mergedGenes_RPKUM.tsv
dataPath='/home/karin/Documents/timeTrajectories/data/RPKUM/combined/'
#Path for 
pathRegulons='/home/karin/Documents/timeTrajectories/data/regulons/by_strain/kN300_mean0std1_log/'

## Used gene sets
Use gene sets that do not have less than 5 or more than 500 genes. Use all ontologies.

In [3]:
# Get all gene sets
gene_sets=list(list_all(organism='44689'))
GENE_SETS_ONTOLOGY=enr.get_gene_sets(gene_set_names=gene_sets, go_slims=True,set_sizes=(5,500))

In [4]:
# For conversion of gene names to EID
genes = pd.read_csv(dataPath + 'mergedGenes_RPKUM.tsv', sep='\t', index_col=0)
all_gene_names= genes[(genes != 0).any(axis=1)].index
NAME_EID=enr.name_genes_entrez(gene_names=all_gene_names, key_entrez=False)
ALL_GENE_NAMES_EID=enr.convert_EID(genes=all_gene_names, name_EID=NAME_EID)

In [5]:
def group_diff_enrichment(data:pd.DataFrame,group:str,padj:float=0.25,min_overlap:int=None,
                          use_annotated_genes:bool=False):
    #Displays only gene sets that have overlap with query greater or equal to min_overlap 
    #For p value and padj calculation uses alll that have overlap >=1 } from gene_set_enrichment
    """
    :param use_annotated_genes: if True use for reference and query  only genes that have at 
    least one gene set annotation
    """
    selected=list(data[data['Cluster']==group].index)
    query_EID=enr.convert_EID(genes=selected, name_EID=NAME_EID)
    print('***  '+group+' selected:',len(selected),'with EID:',len(query_EID))
    
    reference_gene_eids=ALL_GENE_NAMES_EID.copy()
    query_eids=query_EID.copy()
    
    if use_annotated_genes:
        gene_sets_genes=set()
        for gene_set_name, gene_sets in GENE_SETS_ONTOLOGY.items():
            for gene_set in gene_sets:
                gene_sets_genes.update(gene_set.genes)
        reference_gene_eids=set(reference_gene_eids) & gene_sets_genes
        query_eids=set(query_eids) & gene_sets_genes
        print('Ratio of genes annotated with a gene set in reference',
              round(len(reference_gene_eids)/len(ALL_GENE_NAMES_EID),2),
             'and query',round(len(query_eids)/len(query_EID),2))
    
    result=None
    if len(query_eids) > 0:
        enrichment=enr.gene_set_enrichment(query_eids, reference_EID=reference_gene_eids, 
                                                padj_threshold=padj,min_overlap=min_overlap,
                                                gene_sets_ontology=GENE_SETS_ONTOLOGY)
        query_in_enriched=set()
        if len(enrichment)>0:
            enrichment_display=list()
            enrichment= sorted(enrichment, key=lambda data: data.padj)
            for enriched in enrichment:
                query_in_enriched.update(enriched.gene_set.genes & query_eids)
                enrichment_display.append({'Gene set':enriched.gene_set.name,'Ontology':enriched.ontology,
                                           'FDR':"{:.2e}".format(enriched.padj),'N in query':enriched.in_query})
            result=pd.DataFrame(enrichment_display)
    print('Enrichment at FDR: '+str(padj)+' and min query - gene set overlap',str(min_overlap))
    print('N query genes in displayed gene sets:',len(query_in_enriched),'out of', len(query_eids),
          'query genes used for enrichment calculation.')
    display(result)
    return result

## Enrichment reports

In [11]:
save_enrichment=False

### Enrichment for expression_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.pdf

In [6]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2)
    if save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 91 with EID: 91
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 34 out of 91 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,cell wall,"(GO, cellular_component)",8.23e-11,7
1,external encapsulating structure,"(GO, cellular_component)",8.23e-11,7
2,aberrant spore coat morphology,"(Dictybase, Phenotypes)",4.29e-09,6
3,anatomical structure formation involved in mor...,"(GO, biological_process)",8.53e-08,9
4,cell wall organization or biogenesis,"(GO, biological_process)",1.17e-06,5
5,cell differentiation,"(GO, biological_process)",1.85e-06,10
6,anatomical structure development,"(GO, biological_process)",2.58e-04,13
7,abolished cellulose binding,"(Dictybase, Phenotypes)",7.13e-04,2
8,Starch and sucrose metabolism,"(KEGG, Pathways)",8.81e-04,4
9,abolished stalk cell differentiation,"(Dictybase, Phenotypes)",1.06e-02,2


***  C2 selected: 77 with EID: 77
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 13 out of 77 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,cell differentiation,"(GO, biological_process)",2.82e-03,7
1,reproduction,"(GO, biological_process)",6.79e-02,4
2,Starch and sucrose metabolism,"(KEGG, Pathways)",1.01e-01,2
3,anatomical structure development,"(GO, biological_process)",1.01e-01,7
4,cell adhesion,"(GO, biological_process)",1.01e-01,3
5,decreased sporulation,"(Dictybase, Phenotypes)",1.44e-01,2
6,development arrests at mound stage,"(Dictybase, Phenotypes)",1.44e-01,2
7,abolished culmination,"(Dictybase, Phenotypes)",1.66e-01,2
8,delayed aggregation,"(Dictybase, Phenotypes)",2.00e-01,2
9,anatomical structure formation involved in mor...,"(GO, biological_process)",2.00e-01,2


***  C3 selected: 74 with EID: 74
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 24 out of 74 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,kinase activity,"(GO, molecular_function)",1.10e-01,3


***  C20 selected: 20 with EID: 20
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 20 query genes used for enrichment calculation.


None

***  C21 selected: 19 with EID: 19
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 19 query genes used for enrichment calculation.


None

#### Using only genes annotated with at least one gene set 
To find genes that have gene set annotations only gene sets used in enrichment are used (see above). Genes that are in at least one of these gene sets are used for enrichment calculation, both for reference and gene group of interest (query).

In [7]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2,use_annotated_genes=True)
    if  save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30_annotatedOnly_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 91 with EID: 91
Ratio of genes annotated with a gene set in reference 0.32 and query 0.45
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 34 out of 41 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,cell wall,"(GO, cellular_component)",6.81e-10,7
1,external encapsulating structure,"(GO, cellular_component)",6.81e-10,7
2,aberrant spore coat morphology,"(Dictybase, Phenotypes)",2.74e-08,6
3,anatomical structure formation involved in mor...,"(GO, biological_process)",1.03e-06,9
4,cell wall organization or biogenesis,"(GO, biological_process)",5.66e-06,5
5,cell differentiation,"(GO, biological_process)",2.53e-05,10
6,abolished cellulose binding,"(Dictybase, Phenotypes)",1.61e-03,2
7,Starch and sucrose metabolism,"(KEGG, Pathways)",3.46e-03,4
8,anatomical structure development,"(GO, biological_process)",3.50e-03,13
9,abolished stalk cell differentiation,"(Dictybase, Phenotypes)",2.28e-02,2


***  C2 selected: 77 with EID: 77
Ratio of genes annotated with a gene set in reference 0.32 and query 0.29
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 15 out of 22 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,cell differentiation,"(GO, biological_process)",7.48e-04,7
1,reproduction,"(GO, biological_process)",6.08e-02,4
2,anatomical structure development,"(GO, biological_process)",7.24e-02,7
3,cell adhesion,"(GO, biological_process)",8.83e-02,3
4,Starch and sucrose metabolism,"(KEGG, Pathways)",9.08e-02,2
5,decreased sporulation,"(Dictybase, Phenotypes)",1.20e-01,2
6,development arrests at mound stage,"(Dictybase, Phenotypes)",1.20e-01,2
7,abolished culmination,"(Dictybase, Phenotypes)",1.38e-01,2
8,delayed aggregation,"(Dictybase, Phenotypes)",1.69e-01,2
9,anatomical structure formation involved in mor...,"(GO, biological_process)",1.72e-01,2


***  C3 selected: 74 with EID: 74
Ratio of genes annotated with a gene set in reference 0.32 and query 0.42
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 24 out of 31 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",1.08e-15,12
1,nucleolus,"(GO, cellular_component)",8.61e-14,11
2,ribosome biogenesis,"(GO, biological_process)",6.18e-13,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",1.10e-05,4
4,RNA binding,"(GO, molecular_function)",5.66e-04,6
5,rRNA binding,"(GO, molecular_function)",5.45e-03,2
6,protein-containing complex assembly,"(GO, biological_process)",1.42e-02,4
7,cellular component assembly,"(GO, biological_process)",4.41e-02,4
8,helicase activity,"(GO, molecular_function)",5.26e-02,2
9,methyltransferase activity,"(GO, molecular_function)",5.63e-02,2


***  C19 selected: 23 with EID: 23
Ratio of genes annotated with a gene set in reference 0.32 and query 0.26
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 3 out of 6 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,kinase activity,"(GO, molecular_function)",4.71e-02,3


***  C20 selected: 20 with EID: 20
Ratio of genes annotated with a gene set in reference 0.32 and query 0.0
Enrichment at FDR: 0.25 and min query - gene set overlap 2


UnboundLocalError: local variable 'query_in_enriched' referenced before assignment

### Enrichment for mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab

In [8]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2)
    if  save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 96 with EID: 96
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 30 out of 96 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,anatomical structure development,"(GO, biological_process)",2.66e-05,16
1,cell differentiation,"(GO, biological_process)",8.27e-03,7
2,development arrests at loose mound stage,"(Dictybase, Phenotypes)",1.47e-02,3
3,signal transduction,"(GO, biological_process)",1.47e-02,10
4,Glycosaminoglycan degradation,"(KEGG, Pathways)",2.18e-02,2
5,increased slug migration,"(Dictybase, Phenotypes)",2.52e-02,2
6,aberrant aggregation,"(Dictybase, Phenotypes)",2.52e-02,4
7,extracellular region,"(GO, cellular_component)",2.52e-02,6
8,decreased aggregate size,"(Dictybase, Phenotypes)",2.65e-02,3
9,decreased fruiting body size,"(Dictybase, Phenotypes)",3.40e-02,4


***  C2 selected: 89 with EID: 88
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 10 out of 88 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,extracellular region,"(GO, cellular_component)",2.63e-03,7
1,anatomical structure formation involved in mor...,"(GO, biological_process)",1.90e-02,4
2,Starch and sucrose metabolism,"(KEGG, Pathways)",1.02e-01,2
3,cell differentiation,"(GO, biological_process)",1.02e-01,4


***  C3 selected: 87 with EID: 87
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 36 out of 87 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",7.70e-19,12
1,nucleolus,"(GO, cellular_component)",9.74e-17,11
2,ribosome biogenesis,"(GO, biological_process)",1.15e-15,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",8.06e-07,4
4,RNA binding,"(GO, molecular_function)",1.36e-05,6
5,protein-containing complex assembly,"(GO, biological_process)",1.41e-03,4
6,rRNA binding,"(GO, molecular_function)",1.41e-03,2
7,cellular component assembly,"(GO, biological_process)",4.46e-03,4
8,helicase activity,"(GO, molecular_function)",1.68e-02,2
9,methyltransferase activity,"(GO, molecular_function)",1.82e-02,2


***  C19 selected: 20 with EID: 20
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 18 out of 20 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,ribosome,"(GO, cellular_component)",4.29e-33,17
1,Ribosome,"(KEGG, Pathways)",4.70e-33,17
2,structural constituent of ribosome,"(GO, molecular_function)",1.12e-28,15
3,structural molecule activity,"(GO, molecular_function)",1.46e-26,15
4,translation,"(GO, biological_process)",5.80e-22,14
5,rRNA binding,"(GO, molecular_function)",3.78e-06,3
6,RNA binding,"(GO, molecular_function)",1.82e-05,5
7,cytosol,"(GO, cellular_component)",3.81e-05,5
8,ribosome biogenesis,"(GO, biological_process)",3.02e-04,3
9,delayed development,"(Dictybase, Phenotypes)",1.17e-02,2


***  C20 selected: 1 with EID: 1
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 1 query genes used for enrichment calculation.


None

#### Using only genes annotated with at least one gene set

In [9]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2,use_annotated_genes=True)
    if save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 96 with EID: 96
Ratio of genes annotated with a gene set in reference 0.32 and query 0.39
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 30 out of 37 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,anatomical structure development,"(GO, biological_process)",6.78e-05,16
1,cell differentiation,"(GO, biological_process)",2.02e-02,7
2,Glycosaminoglycan degradation,"(KEGG, Pathways)",3.13e-02,2
3,development arrests at loose mound stage,"(Dictybase, Phenotypes)",3.13e-02,3
4,signal transduction,"(GO, biological_process)",3.13e-02,10
5,increased slug migration,"(Dictybase, Phenotypes)",4.18e-02,2
6,aberrant aggregation,"(Dictybase, Phenotypes)",4.18e-02,4
7,decreased aggregate size,"(Dictybase, Phenotypes)",4.36e-02,3
8,extracellular region,"(GO, cellular_component)",4.36e-02,6
9,decreased fruiting body size,"(Dictybase, Phenotypes)",6.04e-02,4


***  C2 selected: 89 with EID: 88
Ratio of genes annotated with a gene set in reference 0.32 and query 0.2
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 10 out of 18 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,extracellular region,"(GO, cellular_component)",6.50e-05,7
1,anatomical structure formation involved in mor...,"(GO, biological_process)",2.90e-03,4
2,cell differentiation,"(GO, biological_process)",2.42e-02,4
3,Starch and sucrose metabolism,"(KEGG, Pathways)",4.25e-02,2


***  C3 selected: 87 with EID: 87
Ratio of genes annotated with a gene set in reference 0.32 and query 0.45
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 34 out of 39 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",3.09e-15,12
1,nucleolus,"(GO, cellular_component)",2.23e-13,11
2,ribosome biogenesis,"(GO, biological_process)",1.47e-12,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",1.76e-05,4
4,RNA binding,"(GO, molecular_function)",9.85e-04,6
5,rRNA binding,"(GO, molecular_function)",7.86e-03,2
6,protein-containing complex assembly,"(GO, biological_process)",2.21e-02,4
7,cellular component assembly,"(GO, biological_process)",6.78e-02,4
8,helicase activity,"(GO, molecular_function)",7.54e-02,2
9,methyltransferase activity,"(GO, molecular_function)",8.06e-02,2


***  C19 selected: 20 with EID: 20
Ratio of genes annotated with a gene set in reference 0.32 and query 0.9
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 18 out of 18 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,ribosome,"(GO, cellular_component)",1.92e-26,17
1,Ribosome,"(KEGG, Pathways)",2.10e-26,17
2,structural constituent of ribosome,"(GO, molecular_function)",1.65e-22,15
3,structural molecule activity,"(GO, molecular_function)",2.13e-20,15
4,translation,"(GO, biological_process)",3.87e-16,14
5,rRNA binding,"(GO, molecular_function)",8.21e-05,3
6,RNA binding,"(GO, molecular_function)",2.30e-03,5
7,cytosol,"(GO, cellular_component)",4.53e-03,5
8,ribosome biogenesis,"(GO, biological_process)",6.00e-03,3
9,delayed development,"(Dictybase, Phenotypes)",8.30e-02,2


***  C20 selected: 1 with EID: 1
Ratio of genes annotated with a gene set in reference 0.32 and query 0.0
Enrichment at FDR: 0.25 and min query - gene set overlap 2


UnboundLocalError: local variable 'query_in_enriched' referenced before assignment